# ArcGIS Online Map and Daya Layer Harvester v. 1.0
This script reads in a CSV file with mapids and writes out select metadata of the maps and constituent data layers to a MySQL table. This python file is also used to populate the table that is later exposed to end users via a webserver middleware script.   

Script requires at least ArcGIS Python API v 1.5.1 or later, pymysql, and some other fun modules.

/ trbaker, January 2019

In [ ]:
# take a map id
from arcgis.gis import GIS
from arcgis.mapping import WebMap
from arcgis.features import FeatureLayer  #used to capture feature class layer description
import csv
import pymysql.cursors  #used to write data to mysql

#initialize
loop=0
mapcount=0
log=""
gis = GIS("https://arcgis.com")

#functions
def dbwriter(objectType,mapId,objectName,url,description,tags='',serviceName=''):
    #clean up some of the data in objectName and description
    objectName=objectName.replace("'", "&#39;")
    objectName=objectName.replace("™", "")   
    description=description.replace("'", "&#39;")
    description=description.replace("™", "") 
    description=description.replace("-", "")
    serviceName=serviceName.replace('\'', '&#39;')
    tags = tags[:149]
    description = description[:500]
    connection = pymysql.connect(host="", user="", password="", db='', cursorclass=pymysql.cursors.DictCursor)
    with connection.cursor() as cursor:
        sql = "insert into <tablename> (objectType,mapId,serviceName,objectName,url,description,tags) values(" + "'" + objectType + "','" + mapId + "','" + serviceName + "','" + objectName + "','" + url + "','" + description + "','" + tags + "')"
        cursor.execute(sql)
    connection.commit()
    return

#read, loop, and process over list of map ids. Mapids are in column A of CSV.     
with open('data/maps.csv') as f:
    reader = csv.reader(f)
    for map in reader: 
        try:
            result = gis.content.get(map[0])
        except:
            log = log + "Map inaccessible,https://www.arcgis.com/home/item.html?id=" + map[0] + ",Map error" + "\r\n"
            continue
        web_map_obj = WebMap(result)
        web_map_obj
        web_map_obj.layers
        #clean up special characters
        if (hasattr(result, 'tags')):
            stags = str(result.tags)
            stags=stags.replace(',', '')
            stags=stags.replace('\'', '')
            stags=stags.replace('[', '')
            stags=stags.replace(']', '')
        else:
            stags=""
        if (hasattr(result, 'snippet')): 
            if result.snippet is not None:
                ssnippet = result.snippet.replace(',', '')
        else:
            ssnippet=""
        dbwriter(objectType='Webmap',mapId=result.itemid,objectName=result.title,url='http://www.maps.arcgis.com/home/webmap/viewer.html?webmap=' + result.itemid,description=ssnippet,tags=stags)    
        #used for screen output only
        mapcount=mapcount+1
        print(str(mapcount) + ': ' + result.title)
        
# LAYERS LISTING IN EACH MAP 
        loop=0  
        for lyr in web_map_obj.layers:
            loop=loop+1
            if (hasattr(lyr, 'snippet')):
                snippet = lyr.snippet
            else:
                snippet=""
            #data services with itemId
            if (hasattr(lyr, 'itemId')) and not (hasattr(lyr, 'url')): 
                if hasattr(lyr, 'url'):
                    dbwriter(objectType='layer',mapId=lyr.itemid,serviceName='',objectName=lyr.title,url=lyr.url,description=ssnippet)    
                else:
                    dbwriter(objectType='layer',mapId=lyr.itemId,serviceName='',objectName=lyr.title,url='http://www.arcgis.com/home/item.html?id=' + lyr.itemId,description=ssnippet)    
                try:
                    gis.content.get(lyr.itemId)
                except:
                    print("Error accessing layer")
            else: #lyr.url path  
                if hasattr(lyr, 'url'): 
                    try:
                        fclyr = FeatureLayer(lyr.url)
                        dbwriter(
                            objectType='layer',
                            mapId=map[0],
                            serviceName=lyr.id,
                            objectName=lyr.title,
                            url=lyr.url,
                            description=fclyr.properties.description)
                    except:
                        dbwriter(
                            objectType='layer',
                            mapId=map[0],
                            serviceName=lyr.id,
                            objectName=lyr.title,
                            url=lyr.url,
                            description='')
print("Complete.")